<a href="https://colab.research.google.com/github/Ahmed-Medhat1911/Customer-Segmentation/blob/main/Customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#@title import Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [12]:
#@title Read the Data
data=pd.read_excel("/content/Online Retail.xlsx")

In [13]:

#@title take copy from the data
data=data.copy()

In [14]:
#@title Show data
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
#@title Show basic insights from the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [16]:
#@title show statistical insights from the data
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [17]:
#@title Data preprocessing



In [18]:
#@title Show missing values
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [19]:
#@title remove missing values rows as i need customer id and description
data=data.dropna()

In [20]:
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [21]:
#@title grouping by description
data.groupby('Description')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False)

,Description,Quantity
3805,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
1771,JUMBO BAG RED RETROSPOT,45066
217,ASSORTED COLOUR BIRD ORNAMENT,35314
3716,WHITE HANGING HEART T-LIGHT HOLDER,34147
2279,PACK OF 72 RETROSPOT CAKE CASES,33409
...,...,...
2537,PINK POODLE HANGING DECORATION,-12
882,CRUK Commission,-16
245,ASSORTED TUTTI FRUTTI ROUND BOX,-24
1066,Discount,-1194


In [22]:
#@title Drop cancelled orders
data = data[data['InvoiceNo'].astype(str).str.upper().str.contains('C') != True]

In [23]:

import pandas as pd

# Assuming your DataFrame is named 'data'
# Replace 'data' with the actual name of your DataFrame

# Convert 'InvoiceDate' to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Calculate the maximum date for each customer
max_date_per_customer = data.groupby('CustomerID')['InvoiceDate'].max()

# Calculate the number of days from the last order to 'today' (11/12/2011)
data['DaysFromLastOrder'] = (pd.to_datetime('2011-12-11') - max_date_per_customer).dt.days

# Calculate the total price for each order
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

# Group by 'CustomerID' and aggregate the sum of total price and the number of unique orders
customer_summary = data.groupby('CustomerID').agg({
    'TotalPrice': 'sum',
    'InvoiceNo': 'nunique'
}).reset_index()

# Rename columns
customer_summary.columns = ['CustomerID', 'Monetary', 'Frequency']

# Merge with the 'DaysFromLastOrder' information
customer_summary = pd.merge(customer_summary, max_date_per_customer.reset_index(), on='CustomerID')
customer_summary['DaysFromLastOrder'] = (pd.to_datetime('2011-12-11') - customer_summary['InvoiceDate']).dt.days

# Drop the 'InvoiceDate' column as it's no longer needed
customer_summary = customer_summary.drop('InvoiceDate', axis=1)

# Rename columns based on your requirement
customer_summary.columns = ['CustomerID', 'Monetary', 'Frequency', 'Recency']

# Use qcut to create quantile-based bins with equal counts
customer_summary['RecencyRating'] = pd.qcut(customer_summary['Recency'], q=5, labels=False, duplicates='drop', retbins=False, precision=3) + 1
customer_summary['FrequencyRating'] = pd.qcut(customer_summary['Frequency'], q=5, labels=False, duplicates='drop', retbins=False, precision=3) + 1
customer_summary['MonetaryRating'] = pd.qcut(customer_summary['Monetary'], q=5, labels=False, duplicates='drop', retbins=False, precision=3) + 1

# Reverse the order for RecencyRating
customer_summary['RecencyRating'] = 6 - customer_summary['RecencyRating']

# Display the result
print(customer_summary)


      CustomerID  Monetary  Frequency  Recency  RecencyRating  \
0        12346.0  77183.60          1      326              1   
1        12347.0   4310.00          7        3              5   
2        12348.0   1797.24          4       76              2   
3        12349.0   1757.55          1       19              4   
4        12350.0    334.40          1      311              1   
...          ...       ...        ...      ...            ...   
4334     18280.0    180.60          1      278              1   
4335     18281.0     80.82          1      181              1   
4336     18282.0    178.05          2        8              5   
4337     18283.0   2094.88         16        4              5   
4338     18287.0   1837.28          3       43              3   

      FrequencyRating  MonetaryRating  
0                   1               5  
1                   4               5  
2                   3               4  
3                   1               4  
4                  

<ipython-input-23-ec39a4cf268e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
<ipython-input-23-ec39a4cf268e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DaysFromLastOrder'] = (pd.to_datetime('2011-12-11') - max_date_per_customer).dt.days
<ipython-input-23-ec39a4cf268e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [24]:
#@title Removing orders with 0 monetary
rfm=customer_summary[customer_summary["Monetary"]>0]

In [25]:
rfm.sample(5)

,CustomerID,Monetary,Frequency,Recency,RecencyRating,FrequencyRating,MonetaryRating
1725,14696.0,2078.95,7,5,5,4,5
1643,14578.0,168.63,1,4,5,1,1
4062,17904.0,191.67,1,22,4,1,1
1482,14364.0,3717.35,8,109,2,4,5
3999,17824.0,298.40,1,52,3,1,2


In [28]:
#@title Calculate RFM Score
customer_summary['RFM Score'] = customer_summary['RecencyRating'].astype(str) + customer_summary['FrequencyRating'].astype(str).astype(str)

# Display the result
print(customer_summary)

      CustomerID  Monetary  Frequency  Recency  RecencyRating  \
0        12346.0  77183.60          1      326              1   
1        12347.0   4310.00          7        3              5   
2        12348.0   1797.24          4       76              2   
3        12349.0   1757.55          1       19              4   
4        12350.0    334.40          1      311              1   
...          ...       ...        ...      ...            ...   
4334     18280.0    180.60          1      278              1   
4335     18281.0     80.82          1      181              1   
4336     18282.0    178.05          2        8              5   
4337     18283.0   2094.88         16        4              5   
4338     18287.0   1837.28          3       43              3   

      FrequencyRating  MonetaryRating RFM Score  
0                   1               5        11  
1                   4               5        54  
2                   3               4        23  
3                  

In [33]:
#@title Segmenting Customers using RFM Score
seg_map={
    r'[1-2][1-2]':'hibernating',
    r'[1-2][3-4]':'at risk',
    r'[1-2][5]':'Can`t loose',
    r'[3][1-2]':'about to sleep',
    r'33' :'need attention',
    r'[3-4][4-5]':'Loyal Customers',
    r'41' :'Promising',
    r'51' :'New Customers',
    r'[4-5][2-3]':'Potential Loyalities',
    r'5[4-5]':'Champions'




}
customer_summary["segment"]=customer_summary["RFM Score"].replace(seg_map,regex=True)


In [34]:
customer_summary

,CustomerID,Monetary,Frequency,Recency,RecencyRating,FrequencyRating,MonetaryRating,RFM Score,segment
0,12346.0,77183.60,1,326,1,1,5,11,hibernating
1,12347.0,4310.00,7,3,5,4,5,54,Champions
2,12348.0,1797.24,4,76,2,3,4,23,at risk
3,12349.0,1757.55,1,19,4,1,4,41,Promising
4,12350.0,334.40,1,311,1,1,2,11,hibernating
...,...,...,...,...,...,...,...,...,...
4334,18280.0,180.60,1,278,1,1,1,11,hibernating
4335,18281.0,80.82,1,181,1,1,1,11,hibernating
4336,18282.0,178.05,2,8,5,1,1,51,New Customers
4337,18283.0,2094.88,16,4,5,4,5,54,Champions
